In [1]:
import sys, os
module_paths = [os.path.abspath('.'), os.path.abspath('..')]
for module_path in module_paths:
    if module_path not in sys.path:
        sys.path.append(module_path)

import numpy as np
import pandas as pd
import scanpy as sc
import networkx as nx

import torch
import torch_geometric as pyg

In [2]:
dataset = "marson" #"sciplex", "L008"

# 1. initial
graph = "grn.pkl"
dimension = 8

In [3]:
'''
# 2. updated
graph = "%s_grn.pkl" % dataset
dimension = 128
'''

'\n# 2. updated\ngraph = "%s_grn.pkl" % dataset\ndimension = 128\n'

In [4]:
adata = sc.read('../datasets/%s_prepped.h5ad' % dataset)
adata.var

,ENSGNM,name,is_sgRNA,mito,highly_variable,means,dispersions,dispersions_norm
ISG15,ENSG00000187608,ISG15,False,False,True,0.464187,0.646769,2.577193
TNFRSF18,ENSG00000186891,TNFRSF18,False,False,True,0.069438,0.918909,6.711458
TNFRSF4,ENSG00000186827,TNFRSF4,False,False,True,0.220937,1.205736,19.591536
SLC35E2A,ENSG00000215790,SLC35E2A,False,False,True,0.054039,0.745175,1.580994
PLCH2,ENSG00000149527,PLCH2,False,False,True,0.002080,0.842152,1.986914
...,...,...,...,...,...,...,...,...
UTY,ENSG00000183878,UTY,False,False,True,0.063287,0.762314,2.413199
TTTY14,ENSG00000176728,TTTY14,False,False,True,0.008703,0.807486,1.762861
KDM5D,ENSG00000012817,KDM5D,False,False,True,0.047676,0.753803,1.807152
TTTY10,ENSG00000229236,TTTY10,False,False,True,0.007920,0.810274,1.819597


In [5]:
nodes = np.array(adata.var.index)
nodes

array(['ISG15', 'TNFRSF18', 'TNFRSF4', ..., 'KDM5D', 'TTTY10', 'EIF1AY'],
      dtype=object)

In [6]:
source_path = "./node/source/%s" % dataset
export_path = "./node/export/%s" % dataset
os.makedirs(source_path, exist_ok=True)
os.makedirs(export_path, exist_ok=True)

In [7]:
np.savetxt(os.path.join(source_path, "genes.txt"),
    np.expand_dims(nodes, 1), fmt="%s"
)

In [8]:
from node.gene2vec import gene2vec

gene2vec(os.path.abspath(source_path), os.path.abspath(export_path), 'txt', dimension=dimension)

2023-03-06 01:45:52.263930
current file genes.txt num: 1 total files 1
2023-03-06 01:45:52.266395
shuffle start 2013
2023-03-06 01:45:52.267589
shuffle done 2013
gene2vec dimension 8 iteration 1 start
gene2vec dimension 8 iteration 1 done
2023-03-06 01:45:52.677112
shuffle start 2013
2023-03-06 01:45:52.678309
shuffle done 2013
gene2vec dimension 8 iteration 2 start
gene2vec dimension 8 iteration 2 done
2023-03-06 01:45:52.849686
shuffle start 2013
2023-03-06 01:45:52.850959
shuffle done 2013
gene2vec dimension 8 iteration 3 start
gene2vec dimension 8 iteration 3 done
2023-03-06 01:45:52.888199
shuffle start 2013
2023-03-06 01:45:52.888933
shuffle done 2013
gene2vec dimension 8 iteration 4 start
gene2vec dimension 8 iteration 4 done
2023-03-06 01:45:52.925776
shuffle start 2013
2023-03-06 01:45:52.926510
shuffle done 2013
gene2vec dimension 8 iteration 5 start
gene2vec dimension 8 iteration 5 done
2023-03-06 01:45:52.962992
shuffle start 2013
2023-03-06 01:45:52.963717
shuffle done 201

In [9]:
nodes_emb = pd.read_csv(os.path.join(export_path, "gene2vec_dim_%s_iter_10.txt") % dimension, sep=" ", index_col=0, header=None)
nodes_emb

,1,2,3,4,5,6,7,8
0,,,,,,,,
TNFAIP8L2,0.007664,-0.054745,0.020132,-0.017255,0.060192,0.014966,-0.055646,0.050988
FAM102A,-0.017653,-0.041764,0.040962,0.046984,0.034170,-0.019739,0.040485,0.042029
CXCR4,-0.004586,0.013479,-0.039426,0.061475,-0.019202,-0.029176,0.010473,-0.044822
EBNA1BP2,-0.024190,0.030303,0.005047,-0.052961,-0.004977,-0.062102,-0.058747,-0.051593
GMNN,0.023840,0.010448,-0.018525,0.032117,0.005792,0.049402,0.033119,-0.013235
...,...,...,...,...,...,...,...,...
TRAV38-1,0.046085,-0.019390,0.041121,0.050590,-0.051420,-0.037592,-0.025264,0.028561
LYZ,-0.061699,-0.025620,0.028061,-0.053301,-0.059312,-0.005600,0.005772,-0.007870
TRBV5-1,-0.033344,0.054313,0.008754,-0.045582,-0.022257,0.012917,-0.054330,-0.060394


In [10]:
nodes_emb_dict = nodes_emb.T.to_dict('list')

In [11]:
import pickle
'''
from src.utils.graph_utils import parse_grn

graph_df = pd.read_parquet('hg19_TFinfo_dataframe_gimmemotifsv5_fpr2_threshold_10_20210630.parquet')
nx_graph = parse_grn(graph_df, 'gene_short_name')

pickle.dump(nx_graph, open('./grn.pkl', 'wb'))
'''
with open(graph, 'rb') as f:
    nx_graph = pickle.load(f)

In [12]:
for n in nodes:
    if not nx_graph.has_node(n):
        nx_graph.add_node(n)
def filter_node(n):
    return n in nodes
nx_graph = nx.subgraph_view(nx_graph, filter_node)

In [13]:
nodes_dict = dict(zip(nodes, range(len(nodes))))

In [14]:
x = []
edge_index = []
for i, node in enumerate(nodes):
    x.append(nodes_emb_dict[node])

    edges = list(nx_graph.in_edges(node))
    edges = [(nodes_dict[n[0]], i) for n in edges]
    edge_index.extend(edges)

In [15]:
x = torch.Tensor(x)
x.size()

torch.Size([2013, 8])

In [16]:
edge_index = torch.LongTensor(edge_index)
edge_index.size()

torch.Size([42563, 2])

In [17]:
grn = pyg.data.Data(x, edge_index.t())
torch.save(grn, '%s_grn_%s.pth' % (dataset, dimension))

In [18]:
torch.load('%s_grn_%s.pth' % (dataset, dimension))

Data(x=[2013, 8], edge_index=[2, 42563])